In [38]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Configurations

In [39]:
letters = "abcdefghijklmnopqrstuvwxyzäüöß"
symbols = " " + letters

symbol_count = len(symbols)
vector_size = 16

word_size = 32


# Data

In [40]:
def new_words_gen(filename, accent_char="'"):
    with open(filename, 'r', encoding='utf8') as file:
        for line in file:
            accent_word = line.strip().lower()
            
            accent = accent_word.find(accent_char)
            if accent == -1:
                continue
            
            word = accent_word[:accent] + accent_word[accent + 1:]
            if word.find(accent_char) != -1:
                # print("skipped word \""+accent_word+"\" because of multiple accents.")
                continue
            
            if len(word) > word_size:
                # print("skipped word \""+accent_word+"\" because it is to long.")
                continue
            
            yield word, accent


next(new_words_gen('./data/data.txt'))


('aachen', 2)

In [55]:
def word_to_indices(word):
    indices = []
    for c in word:
        index = symbols.find(c)
        if index == -1:
            continue
        indices.append(index)
    indices.extend([0] * (word_size - len(indices)))
    return indices

def new_indices_gen(words_gen):
    for word, accent in words_gen:
        indices = word_to_indices(word)
        yield indices, accent


print(next(new_indices_gen(new_words_gen('./data/data.txt'))))

([1, 1, 3, 8, 5, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2)


In [42]:
indices, labels = zip(*new_indices_gen(new_words_gen('./data/data.txt')))
indices = np.array(indices)
labels = np.array(labels)
dataset_size = len(indices)

dataset = tf.data.Dataset.from_tensor_slices((indices, labels))
del indices, labels

train_set_size = int(dataset_size * 0.7)
dataset = dataset.shuffle(1024)
train_dataset = dataset.take(train_set_size)
valid_dataset = dataset.skip(train_set_size)
del dataset, train_set_size


# Letter to Vectors

In [43]:
class LetterToVector(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.vectors = self.add_weight('vectors', (symbol_count, vector_size), tf.float32)
    
    def call(self, inputs, **kwargs):
        return tf.gather(self.vectors, inputs, axis=0, batch_dims=1)


# LetterToVector()(tf.random.uniform((10, word_size), 0, symbol_count, tf.int32)).shape


# BRCNN

In [44]:
class BRCNN(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.forward_cell = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='tanh', input_shape=[vector_size * 2]),
            tf.keras.layers.Dense(16, activation='tanh'),
        ])
        self.backward_cell = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='tanh', input_shape=[vector_size * 2]),
            tf.keras.layers.Dense(16, activation='tanh'),
        ])
        self.union_cell = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='tanh', input_shape=[16 * 2]),
            tf.keras.layers.Dense(16, activation='tanh'),
        ])
    
    def call(self, inputs, **kwargs):
        vectors = tf.unstack(inputs, word_size, axis=-2)
        
        forward_outputs = []
        forward_memory = tf.zeros([tf.shape(inputs)[0], 16])
        for vector in vectors:
            cell_input = tf.concat([vector, forward_memory], axis=-1)
            forward_memory = self.forward_cell(cell_input)
            forward_outputs.append(forward_memory)
        
        backward_outputs = []
        backward_memory = tf.zeros([tf.shape(inputs)[0], 16])
        for vector in reversed(vectors):
            cell_input = tf.concat([vector, backward_memory], axis=-1)
            backward_memory = self.backward_cell(cell_input)
            backward_outputs.append(backward_memory)
        
        union_outputs = []
        for fo, bo in zip(forward_outputs, reversed(backward_outputs)):
            union_input = tf.concat([fo, bo], axis=-1)
            union_output = self.union_cell(union_input)
            union_outputs.append(union_output)
        
        union_outputs = tf.stack(union_outputs, axis=-2)
        return union_outputs


In [45]:
class Model(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ltv = LetterToVector()
        self.brcnn = BRCNN()
        self.scorer = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='tanh'),
            tf.keras.layers.Dense(1)])
    
    def call(self, inputs, training=None, mask=None):
        vectors = self.ltv(inputs)
        vectors = self.brcnn(vectors)
        score = self.scorer(vectors)[..., 0]
        prob = tf.nn.softmax(score, axis=-1)
        return prob


In [46]:
model = Model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [47]:
model.fit(
    x=train_dataset.batch(64),
    validation_data=valid_dataset.batch(64),
    epochs=100,
    callbacks=[
        tf.keras.callbacks.TensorBoard(),
        # tf.keras.callbacks.EarlyStopping(),
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.ModelCheckpoint('./logs/weights_epoch{epoch}_vloss{val_loss}.hdf5', save_best_only=True)
    ]
)

Train for 105 steps, validate for 45 steps
Epoch 1/100
105/105 [==============================] - 14s 135ms/step - loss: 1.0067 - sparse_categorical_accuracy: 0.7470 - val_loss: 0.3982 - val_sparse_categorical_accuracy: 0.8931

In [65]:
words = [ "Anhaltend","anlaufgeschützt" ]
indices = [word_to_indices(word) for word in words]

probs = model.predict(indices)
positions = np.argmax(probs,axis=-1)

for word, position in zip(words, positions):
    print(word[:position+1]+"'"+word[position+1:])


A'nhaltend
an'laufgeschützt
